Microsoft : Classifying Cybersecurity Incidents with Machine Learning

In [1]:
# importing dependencies

from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.ensemble import  RandomForestClassifier
from sklearn.model_selection import train_test_split

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, roc_curve, auc
from sklearn.svm import SVC

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv
from itertools import cycle


## **Data Preprocessing**

In [2]:
# inspite of getting errored in loading large data my pc gets hanged

df1= pd.read_csv(r'/content/GUIDE_Test.csv')


In [ ]:
df1.columns

In [ ]:
# gettinng information of dataframe
df1.info()


In [3]:
# drop the unnecessary variables

df1.drop(columns=['EmailClusterId','Id'],inplace=True)


In [ ]:
# to know about the nulls in the column

ax=df1.isnull().sum()

plt.figsize=(5,20)

ax.plot(kind='bar',color = 'black',width=1)
plt.xlabel(' columns')
plt.ylabel('null values in it')
plt.title('Null value counts by column')

plt.show()



In [ ]:
# getting info of null values
col_nulls=df1.columns[df1.isnull().any()]

df1[col_nulls].info()

In [ ]:
df1['IncidentGrade'].value_counts().plot(kind='hist',x=df1['IncidentGrade'].value_counts(),y=df1['IncidentGrade'].value_counts())

In [ ]:
# plotting the counts of targat variable by the tempelate code from gpt

plt.figure(figsize=(8, 6))
sns.countplot(x=df1['IncidentGrade'], order=df1['IncidentGrade'].value_counts().index)
plt.title('Value Counts of Feature')
plt.xlabel('Feature Names')
plt.ylabel('Count')
for p in plt.gca().patches:
    plt.gca().annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()),
                       ha='center', va='bottom', fontsize=10)
plt.show()


In [6]:
df1['MitreTechniques'] = df1['MitreTechniques'].astype(str)

In [7]:
df = df1.to_numpy()

In [8]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='most_frequent')
df = imputer.fit_transform(df)

In [9]:
df = pd.DataFrame(df, columns=df1.columns)

In [ ]:

ax=df.isnull().sum()

plt.figsize=(5,20)

ax.plot(kind='bar',color = 'black',width=1)
plt.xlabel(' columns')
plt.ylabel('null values in it')
plt.title('Null value counts by column')

plt.show()


In [ ]:
df1['AntispamDirection'].unique()

In [10]:
# encoding necessary column
from sklearn.preprocessing import LabelEncoder,StandardScaler

# Apply LabelEncoder to each column
label_encoder = LabelEncoder()
df = df.apply(lambda col: label_encoder.fit_transform(col) if col.dtypes == 'object' else col)


In [11]:
# splitting data into features and labels

x = df.drop(columns= 'IncidentGrade')
y = df['IncidentGrade']

In [ ]:
# making the data to numpy array to get the faster performance

#x = np.array(x).astype(np.float64)
#y = np.array(y).astype(np.float64)

In [12]:
# splitting data for training models

from sklearn.model_selection import train_test_split

x_train,x_test, y_train,y_test =train_test_split(x,y,test_size=0.3,random_state=42)

In [13]:
from sklearn.preprocessing import MinMaxScaler

scaler =StandardScaler()
xtrscaled_data = scaler.fit_transform(x_train)
xtsscaled_data = scaler.transform(x_test)

In [14]:
# Importing the oversampling model from the imblearn package
from imblearn.over_sampling import RandomOverSampler
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier


# Apply RandomOverSampler to the training data
RSampler = RandomOverSampler(random_state=42)
x_resampled, y_resampled = RSampler.fit_resample(X=xtrscaled_data, y=y_train)


In [ ]:
# plotting the counts of targat variable by the tempelate code from gpt

plt.figure(figsize=(8, 6))
sns.countplot(x=y_resampled, order=y_resampled.value_counts().index)
plt.title('Value Counts of Feature')
plt.xlabel('Feature Names')
plt.ylabel('Count')
for p in plt.gca().patches:
    plt.gca().annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()),
                       ha='center', va='bottom', fontsize=10)
plt.show()

In [ ]:
y_resampled.isnull()

In [ ]:
i=pd.DataFrame(y_resampled)
i

In [39]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier


# Initializing the parameters for model ensembling
lr_model = LogisticRegression()
lr_params = {
    'C':[0.001,0.1,1,10,100],
    'penalty':['l2'],
    'solver':['lbfgs','liblinear']
}

dc_model = DecisionTreeClassifier()
dc_params = {
    'max_depth':[3,5,10,100],
    'min_samples_split':[2,5,10],
    'min_samples_leaf':[1,2,4],
    'criterion':['gini','entropy']
}

knn_model = KNeighborsClassifier()
knn_params = {
    'n_neighbors':[3,5,10],
    'weights':['uniform','distance'],
    'metric':['euclidean','manhattan']
}


rf_model = RandomForestClassifier()
rf_params = {
    'n_estimators': [100, 200, 500],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

In [40]:


lr_grid = GridSearchCV(lr_model,lr_params,cv=3,scoring='accuracy')
dc_grid = GridSearchCV(dc_model,dc_params,cv=3,scoring='accuracy')
knn_grid = GridSearchCV(knn_model,knn_params,cv=3,scoring='accuracy')

rf_grid = GridSearchCV(rf_model,rf_params,cv=3,scoring='accuracy')

In [ ]:
lr_grid.fit(x_resampled,y_resampled)
dc_grid.fit(x_resampled,y_resampled)
knn_grid.fit(x_resampled,y_resampled)
knn_grid.fit(x_resampled,y_resampled)

In [ ]:
dcgridpred = dc_grid.predict(x_test)
accuracy_score(dcgridpred,y_test)

0.7879799666110183

In [ ]:
lrpred = lr_grid.predict(x_test)
accuracy_score(lrpred,y_test)

0.5826377295492488

In [ ]:
knnpred = knn_grid.predict(x_test)
accuracy_score(knnpred,y_test)

0.6585976627712855

In [ ]:
print('best parameter for logistic regression model',lr_grid.best_params_)
print('best score for logistic regression',lr_grid.best_score_)

In [ ]:
print('best parameter for decision tree classifier',dc_grid.best_params_)
print('best score for decision tree classifier',dc_grid.best_score_)

In [ ]:
print('best parameter for knn classifier model',knn_grid.best_params_)
print('best score for knn classifier model',knn_grid.best_score_)

In [12]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
bc = BaggingClassifier(estimator=RandomForestClassifier(),
                       n_estimators=5,
                       random_state = 0)


In [13]:
bc.fit(x_resampled,y_resampled)

BaggingClassifier(estimator=RandomForestClassifier(), n_estimators=5,
                  random_state=0)

In [17]:
y_r_pred = bc.predict(xtsscaled_data)

In [37]:
print(f'Accuracy Score : {accuracy_score(y_r_pred,y_test)}')
print(f'Recall_score : {recall_score(y_r_pred,y_test,average = "weighted")}')
print(f'Precision_score : {precision_score(y_r_pred,y_test,average = "micro")}')
print(f'F1_score : {f1_score(y_r_pred,y_test,average = "macro")}')
print(f'Precision_score : {precision_score(y_r_pred,y_test,average = "micro")}')
print(f'F1_score : {f1_score(y_r_pred,y_test,average = "macro")}')

Accuracy Score : 0.8972640032668617
Recall_score : 0.8972640032668617
Precision_score : 0.8972640032668617
F1_score : 0.8931170072184397
Precision_score : 0.8972640032668617
F1_score : 0.8931170072184397


In [19]:
import joblib
joblib.dump(bc,'bagging_classifier.pkl')

['bagging_classifier.pkl']

In [36]:


from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier  # Default base estimator for AdaBoostClassifier

# Create a RandomForestClassifier instance
rf_model = RandomForestClassifier(n_estimators=5, random_state=42)



rf_model.fit(x_resampled, y_resampled)

RandomForestClassifier(n_estimators=5, random_state=42)

In [16]:
y_pred = rf_model.predict(xtsscaled_data)

In [38]:
# Calculate individual metrics with appropriate averaging
metrics = {
    "Accuracy": accuracy_score(y_test, y_pred),
    "Precision (Macro)": precision_score(y_test, y_pred, average="macro"),
    "Recall (Macro)": recall_score(y_test, y_pred, average="macro"),
    "F1-Score (Macro)": f1_score(y_test, y_pred, average="macro")
}

# Display as a report
for metric, value in metrics.items():
    print(f"{metric} of RandomForestClassifier: {value:.2f}")


Accuracy of RandomForestClassifier: 0.90
Precision (Macro) of RandomForestClassifier: 0.90
Recall (Macro) of RandomForestClassifier: 0.89
F1-Score (Macro) of RandomForestClassifier: 0.89


In [20]:
import joblib

joblib.dump(rf_model,'rf_model.pkl')

['rf_model.pkl']